In [8]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath('../../'))
from mcda.core.functions import Threshold
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.electre.credibility import get_criteria_counts, credibility_electre_iv
from mcda.electre.outranking.ranking import  net_flow_score
from mcda.electre.outranking import crisp_cut

In [9]:
alternative_names = ["ITA", "BEL", "GER", "AUT", "FRA"]
criteria_names = ["Power", "Safety", "Price"]

performance_table = pd.DataFrame([
    [90, 4, 600],
    [58, 0, 200],
    [66, 7, 400],
    [74, 8, 800],
    [98, 6, 800],
], index=alternative_names, columns=criteria_names)
performance_table

,Power,Safety,Price
ITA,90,4,600
BEL,58,0,200
GER,66,7,400
AUT,74,8,800
FRA,98,6,800


In [10]:
scales = pd.Series([
    QuantitativeScale(0, 100),
    QuantitativeScale(0, 10),
    QuantitativeScale(0, 1000, PreferenceDirection.MIN),
], index=criteria_names)

indifference = pd.Series([
    Threshold(0, 4),
    Threshold(0, 1),
    Threshold(0, 100),
 ], index=criteria_names)

preference = pd.Series([
    Threshold(0, 12),
    Threshold(0, 2),
    Threshold(0, 200),
], index=criteria_names)

veto = pd.Series([
    Threshold(0, 28),
    Threshold(0, 8),
    Threshold(0, 600),
], index=criteria_names)

In [11]:
credibility = credibility_electre_iv(performance_table, scales, indifference, preference, veto)
credibility

,ITA,BEL,GER,AUT,FRA
ITA,1.0,0.2,0.0,0.2,0.2
BEL,0.0,1.0,0.2,0.0,0.0
GER,0.2,0.2,1.0,0.6,0.0
AUT,0.0,0.2,0.0,1.0,0.2
FRA,0.2,0.2,0.2,0.2,1.0


Net Flow Score calculated from credibility table

In [12]:
net_flow_score(credibility)

GER    0.6
FRA    0.4
ITA    0.2
BEL   -0.6
AUT   -0.6
dtype: float64

In [13]:
crisp_table = crisp_cut(credibility, 0.5)
crisp_table

,ITA,BEL,GER,AUT,FRA
ITA,True,False,False,False,False
BEL,False,True,False,False,False
GER,False,False,True,True,False
AUT,False,False,False,True,False
FRA,False,False,False,False,True


Net Flow Score calculated from crisp outranking table

In [14]:
net_flow_score(crisp_table)

GER    1
ITA    0
BEL    0
FRA    0
AUT   -1
dtype: int64